## SEIS 764-01 Assignment 10
**Jamie Boehme**

1 . Is your experiment M:1 or M:M? **My experiment is M:1**

2 . Number of sequences, sequence length, number of predictors in each sequence. 
- Number of sequences: 23
- Sequence length: 3
- Number of predictors: 3

3 . Do you have fixed sequence length for all sequences or variable sequence length? **Fixed sequence length of 1**

4 . Do you pad your data? If yes, how do you pad your data? **I did not need to pad my data as all variables had a fixed sequence length of 1. If I had a variable sequence length, I would seek to pad the data**

5 . What is your batch size? **Batch size is equal to 1**

10 . Is your LSTM one-directional or bi-directional? **one-directional**

11 . Please list your LSTM/GRU architecture / configuration. **See model summary below, also annotated with Question 11**

12 . In each experiment, what is the accuracy of your predictions when feeding all your training data/sequences into the final trained LSTM / GRU? **Accuracy equal to 96%**

13 . Please list all (X, Y) pairs in your training records/sequences in each experiment.  **See X and Y below, also annotated with Question 13**

14 . How long does it take to train your model in each experiment? On CPU or GPU? **CPU: 2min, 48 seconds**

Import libraries for processing

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, MaxPooling2D, BatchNormalization, LSTM
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils
from keras.utils import np_utils
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, accuracy_score, roc_curve, auc
import matplotlib.pyplot as plt

!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 19.2 MB/s 
time: 477 µs (started: 2022-11-22 04:46:31 +00:00)


In [2]:
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"


# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))


time: 823 µs (started: 2022-11-22 04:46:31 +00:00)


13 . Please list all (X, Y) pairs in your training records/sequences in each experiment. (starting data)

In [3]:

# prepare the dataset of input to output pairs encoded as integers
seq_length = 3
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
	seq_in = alphabet[i:i + seq_length]
	seq_out = alphabet[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
	print(seq_in, '->', seq_out)

ABC -> D
BCD -> E
CDE -> F
DEF -> G
EFG -> H
FGH -> I
GHI -> J
HIJ -> K
IJK -> L
JKL -> M
KLM -> N
LMN -> O
MNO -> P
NOP -> Q
OPQ -> R
PQR -> S
QRS -> T
RST -> U
STU -> V
TUV -> W
UVW -> X
VWX -> Y
WXY -> Z
time: 5.91 ms (started: 2022-11-22 04:46:31 +00:00)


In [4]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (len(dataX), seq_length, 1))

time: 665 µs (started: 2022-11-22 04:46:31 +00:00)


In [5]:
X.shape

(23, 3, 1)

time: 2.91 ms (started: 2022-11-22 04:46:31 +00:00)


In [6]:
X = X / float(len(alphabet))

time: 695 µs (started: 2022-11-22 04:46:31 +00:00)


In [7]:
y = np_utils.to_categorical(dataY)

time: 4.44 ms (started: 2022-11-22 04:46:31 +00:00)


13 . Please list all (X, Y) pairs in your training records/sequences in each experiment. (training data)

In [8]:
print(X,y)

[[[0.        ]
  [0.03846154]
  [0.07692308]]

 [[0.03846154]
  [0.07692308]
  [0.11538462]]

 [[0.07692308]
  [0.11538462]
  [0.15384615]]

 [[0.11538462]
  [0.15384615]
  [0.19230769]]

 [[0.15384615]
  [0.19230769]
  [0.23076923]]

 [[0.19230769]
  [0.23076923]
  [0.26923077]]

 [[0.23076923]
  [0.26923077]
  [0.30769231]]

 [[0.26923077]
  [0.30769231]
  [0.34615385]]

 [[0.30769231]
  [0.34615385]
  [0.38461538]]

 [[0.34615385]
  [0.38461538]
  [0.42307692]]

 [[0.38461538]
  [0.42307692]
  [0.46153846]]

 [[0.42307692]
  [0.46153846]
  [0.5       ]]

 [[0.46153846]
  [0.5       ]
  [0.53846154]]

 [[0.5       ]
  [0.53846154]
  [0.57692308]]

 [[0.53846154]
  [0.57692308]
  [0.61538462]]

 [[0.57692308]
  [0.61538462]
  [0.65384615]]

 [[0.61538462]
  [0.65384615]
  [0.69230769]]

 [[0.65384615]
  [0.69230769]
  [0.73076923]]

 [[0.69230769]
  [0.73076923]
  [0.76923077]]

 [[0.73076923]
  [0.76923077]
  [0.80769231]]

 [[0.76923077]
  [0.80769231]
  [0.84615385]]

 [[0.80769231

In [9]:
# create and fit the model
X_dim = 1
seq_length= 3
model = Sequential()
model.add(LSTM(9, input_shape=(seq_length, X_dim), return_sequences=False))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=1500, batch_size=1, verbose=1)

Epoch 1/1500
23/23 [==============================] - 8s 4ms/step - loss: 3.2701 - accuracy: 0.0000e+00
Epoch 2/1500
23/23 [==============================] - 0s 3ms/step - loss: 3.2611 - accuracy: 0.0435
Epoch 3/1500
23/23 [==============================] - 0s 3ms/step - loss: 3.2566 - accuracy: 0.0435
Epoch 4/1500
23/23 [==============================] - 0s 4ms/step - loss: 3.2522 - accuracy: 0.0435
Epoch 5/1500
23/23 [==============================] - 0s 3ms/step - loss: 3.2471 - accuracy: 0.0435
Epoch 6/1500
23/23 [==============================] - 0s 4ms/step - loss: 3.2435 - accuracy: 0.0435
Epoch 7/1500
23/23 [==============================] - 0s 4ms/step - loss: 3.2383 - accuracy: 0.0435
Epoch 8/1500
23/23 [==============================] - 0s 4ms/step - loss: 3.2346 - accuracy: 0.0435
Epoch 9/1500
23/23 [==============================] - 0s 3ms/step - loss: 3.2296 - accuracy: 0.0435
Epoch 10/1500
23/23 [==============================] - 0s 3ms/step - loss: 3.2251 - accuracy: 0.

time: 2min 48s (started: 2022-11-22 04:46:31 +00:00)


11. Please list your LSTM/GRU architecture / configuration.

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 9)                 396       
                                                                 
 dense (Dense)               (None, 26)                260       
                                                                 
Total params: 656
Trainable params: 656
Non-trainable params: 0
_________________________________________________________________
time: 39.6 ms (started: 2022-11-22 04:49:19 +00:00)


In [11]:
# evaluate predictions from training
Y_score_train = model.predict(X, verbose=0)
Y_hat_train = np.argmax(Y_score_train, axis=-1)   
print("Classification Report: \n \n", classification_report(dataY, Y_hat_train))
print("Confusion matrix: \n \n", confusion_matrix(dataY, Y_hat_train))

Classification Report: 
 
               precision    recall  f1-score   support

           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         1
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         1
          14       0.00      0.00      0.00         1
          15       0.50      1.00      0.67         1
          16       1.00      1.00      1.00         1
          17       1.00      1.00      1.00         1
          18       1.00      1.00      1.00         1
          19       1.00      1.00      1.00         1


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
